In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
num_samples = 44100

In [3]:
input1 = linspace(-1, 1, num_samples)
input2 = linspace(1, -1, num_samples)

In [4]:
!cat 01_simple.stride

use DesktopAudio version 1.0

signal Accum {}

loop Loop {
	ports: [		
	]
	blocks: [
		signal Counter {}
		switch CounterReset {}
	]
	streams: [
		Accum + 2 >> Accum;
		Counter + 1 >> Counter;
		[Counter, 10] >> Equal() >> CounterReset;
	]
	terminateWhen: CounterReset
}


on >> Loop();
Accum >> AudioOut[1];



In [5]:
out_text = ''

accum = 0

for val1, val2 in zip(input1, input2):
    for i in range(10):
        accum += 2
    out_text += str(accum) + "\n0.0\n"

out_file = open("01_simple.expected", "w")
out_file.write(out_text[:-1])
pass

In [6]:
!cat 02_loop_in_module.stride

use DesktopAudio version 1.0

module LoopModule {
	ports: [
		mainOutputPort OutputPort { block: Output }
	]
	blocks:  [
		signal Accumulator { default: 0}
		loop Accumulate {
			ports: [
				mainInputPort InputPort {
					block: Input
				}
			]
			blocks: [
				signal Counter {default: 0}
				switch CounterReset {}
			]
			streams: [
				Accumulator + 1.5 >> Accumulator;
				Counter + 1 >> Counter;
				[Counter, 10] >> Equal() >> CounterReset;
			]
			terminateWhen: CounterReset
		}
	]
	streams: [
		on >> Accumulate();
		Accumulator >> Output;
	]
}

LoopModule() >> AudioOut[1];



In [7]:
out_text = ''

counter = 0
accum = 0

for val1, val2 in zip(input1, input2):
    for i in range(10):
        accum += 1.5
    out_text += str(accum) + "\n0.0\n"

out_file = open("02_loop_in_module.expected", "w")
out_file.write(out_text[:-1])
pass

In [8]:
!cat 03_list_input_in_module.stride

use DesktopAudio version 1.0

signal Accum {}
switch ResetOuter { }
signal OuterCounter { reset: ResetOuter}

module Sum {
	ports: [
		mainOutputPort OutputPort { block: Output }
		mainInputPort InputPort { block: Input }
	]
	blocks:  [
		signal Input[3] { domain: Output.domain }
		signal Accumulator { default: 0 reset: ResetAccumulator}
		switch ResetAccumulator {}
		signal Counter {default: 1}
		loop Accumulate {
			ports: [	
				mainOutputPort OutputPort { block: Output }
			]
			blocks: [
				switch CounterReset {}
			]
			streams: [
				Accumulator + Input[Counter] >> Accumulator;
				Counter + 1 >> Counter;
				[Counter, 3] >> Greater() >> CounterReset;
			]
			terminateWhen: CounterReset
		}
	]
	streams: [
		Input >> Accumulate();
	]
}

[AudioIn[1], AudioIn[2], 7] >> Sum() >> Value;

